In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
train.head(2)

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)


In [4]:
train.count()

tweet_id                  10980
airline_sentiment         10980
airline                   10980
airline_sentiment_gold       31
name                      10980
negativereason_gold          24
retweet_count             10980
text                      10980
tweet_coord                 776
tweet_created             10980
tweet_location             7430
user_timezone              7403
dtype: int64

In [5]:
# Dropping Useless Columns

train.drop('tweet_id', inplace = True, axis = 1)
train.drop('airline_sentiment_gold', inplace = True, axis = 1)
train.drop('negativereason_gold', inplace = True, axis = 1)
train.drop('tweet_coord', inplace = True, axis = 1)
train.drop('tweet_created',inplace = True, axis = 1)
train.drop('tweet_location',inplace = True, axis = 1)
train.drop('user_timezone',inplace = True, axis = 1) #can be used for suspicious users

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10980 entries, 0 to 10979
Data columns (total 5 columns):
airline_sentiment    10980 non-null object
airline              10980 non-null object
name                 10980 non-null object
retweet_count        10980 non-null int64
text                 10980 non-null object
dtypes: int64(1), object(4)
memory usage: 429.0+ KB


In [7]:
train.head(2)

,airline_sentiment,airline,name,retweet_count,text
0,negative,Southwest,ColeyGirouard,0,"@SouthwestAir I am scheduled for the morning, ..."
1,positive,Southwest,WalterFaddoul,0,@SouthwestAir seeing your workers time in and ...


### Filling NaN values

In [8]:
mood_count = train['airline_sentiment'].value_counts()
mood_count

negative    6851
neutral     2327
positive    1802
Name: airline_sentiment, dtype: int64

In [9]:
train['airline'].value_counts()

United            2928
US Airways        2152
American          2078
Southwest         1817
Delta             1639
Virgin America     366
Name: airline, dtype: int64

In [10]:
train.isnull().sum()

airline_sentiment    0
airline              0
name                 0
retweet_count        0
text                 0
dtype: int64

In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['airline_sentiment'] = le.fit_transform(train['airline_sentiment'])
train.head(2)

,airline_sentiment,airline,name,retweet_count,text
0,0,Southwest,ColeyGirouard,0,"@SouthwestAir I am scheduled for the morning, ..."
1,2,Southwest,WalterFaddoul,0,@SouthwestAir seeing your workers time in and ...


In [12]:
import nltk
from nltk.corpus import stopwords

In [13]:
def tweet_to_words(raw_tweet):

    words = raw_tweet.lower().split()                             
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops] 
    return( " ".join( meaningful_words )) 

In [14]:
train['clean_tweet'] = train['text'].apply(tweet_to_words)

test['clean_tweet'] = test['text'].apply(tweet_to_words)

In [15]:
train_clean_tweet=[]
for tweet in train['clean_tweet']:
    train_clean_tweet.append(tweet)

test_clean_tweet=[]
for tweet in test['clean_tweet']:
    test_clean_tweet.append(tweet)

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer(analyzer = "word")
train_features= v.fit_transform(train_clean_tweet)
test_features= v.transform(test_clean_tweet)

In [17]:
from sklearn.naive_bayes import MultinomialNB

In [18]:
nb = MultinomialNB()

In [19]:
nb.fit(train_features,train['airline_sentiment'])
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(nb.predict(train_features),train['airline_sentiment'])
print(accuracy)

0.854189435337


In [20]:
test_pred = nb.predict(test_features)
test_pred = le.inverse_transform(test_pred)
print(test_pred)
np.savetxt("test_pred.csv",test_pred,fmt='%s')

['negative' 'positive' 'positive' ..., 'negative' 'negative' 'negative']
